### Importing Necessary Libraries

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_models import ChatOllama

_ = load_dotenv()

### Setting Environment Variables

In [2]:
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")

### Initializing GraphCypherQAChain

In [9]:
graph = Neo4jGraph()
cypher_llm = ChatOpenAI(model="gpt-4-turbo")
qa_llm = ChatOpenAI(model="gpt-4-turbo")
chain = GraphCypherQAChain.from_llm(graph=graph, llm=cypher_llm, verbose=True, validate_cypher=True)

### Example Queries

In [10]:
response = chain.invoke({"query": "Which games have same publisher in the year 2003?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2003 AND g2.year = 2003 AND g1 <> g2
RETURN g1.name AS Game1, g2.name AS Game2, p.name AS Publisher

Full Context:
[{'Game1': 'Super Mario Bros. 3', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Pokémon Colosseum', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario & Luigi: Superstar Saga', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario Party 5', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Donkey Kong Country', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Kirby Air Ride', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Pokemon Pinball: Ruby & Sapphire', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario Golf: Toadstool To

{'query': 'Which games have same publisher in the year 2003?',
 'result': 'In the year 2003, the games that have the same publisher, Nintendo, are Super Mario Bros. 3, Pokémon Colosseum, Mario & Luigi: Superstar Saga, Mario Party 5, Donkey Kong Country, Kirby Air Ride, Pokemon Pinball: Ruby & Sapphire, Mario Golf: Toadstool Tour, Final Fantasy: Crystal Chronicles, and WarioWare, Inc.: Mega MicroGame$.'}

In [11]:
response = chain.invoke({"query": "Find pairs of games published by the same publisher in the year 2003 and their total combined global sales."})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2003 AND g2.year = 2003 AND g1 <> g2
RETURN p.name AS Publisher, g1.name AS Game1, g2.name AS Game2, g1.global_sales + g2.global_sales AS TotalGlobalSales

Full Context:
[{'Publisher': 'Nintendo', 'Game1': 'Super Mario Bros. 3', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 12.15}, {'Publisher': 'Nintendo', 'Game1': 'Pokémon Colosseum', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 9.49}, {'Publisher': 'Nintendo', 'Game1': 'Mario & Luigi: Superstar Saga', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 9.120000000000001}, {'Publisher': 'Nintendo', 'Game1': 'Mario Party 5', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 9.030000000000001}, {'Publisher': 'Nintendo', 'Game1': 'Donkey Kong Country', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 8.99}, {'Publisher': 'Nintendo',

{'query': 'Find pairs of games published by the same publisher in the year 2003 and their total combined global sales.',
 'result': 'The pairs of games published by Nintendo in the year 2003 along with their total combined global sales are as follows:\n\n1. "Super Mario Bros. 3" and "Mario Kart: Double Dash!!" with total global sales of 12.15 million.\n2. "Pokémon Colosseum" and "Mario Kart: Double Dash!!" with total global sales of 9.49 million.\n3. "Mario & Luigi: Superstar Saga" and "Mario Kart: Double Dash!!" with total global sales of 9.12 million.\n4. "Mario Party 5" and "Mario Kart: Double Dash!!" with total global sales of 9.03 million.\n5. "Donkey Kong Country" and "Mario Kart: Double Dash!!" with total global sales of 8.99 million.\n6. "Kirby Air Ride" and "Mario Kart: Double Dash!!" with total global sales of 8.57 million.\n7. "Pokemon Pinball: Ruby & Sapphire" and "Mario Kart: Double Dash!!" with total global sales of 8.51 million.\n8. "Mario Golf: Toadstool Tour" and "Mari

In [14]:
response = chain.invoke({"query": "Find the most successful publisher in 2008 in terms of global sales and list their top three games"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:Game)-[:PUBLISHED_BY]->(p:Publisher)
WHERE g.year = 2008
WITH p.name AS Publisher, SUM(g.global_sales) AS TotalSales
ORDER BY TotalSales DESC
LIMIT 1
WITH Publisher
MATCH (g:Game)-[:PUBLISHED_BY]->(p:Publisher {name: Publisher})
WHERE g.year = 2008
RETURN p.name AS Publisher, g.name AS Game, g.global_sales AS GlobalSales
ORDER BY g.global_sales DESC
LIMIT 3

Full Context:
[{'Publisher': 'Nintendo', 'Game': 'Mario Kart Wii', 'GlobalSales': 35.82}, {'Publisher': 'Nintendo', 'Game': 'Super Smash Bros. Brawl', 'GlobalSales': 13.04}, {'Publisher': 'Nintendo', 'Game': 'Pokémon Platinum Version', 'GlobalSales': 7.84}]

> Finished chain.


{'query': 'Find the most successful publisher in 2008 in terms of global sales and list their top three games',
 'result': 'Nintendo is the most successful publisher in 2008 in terms of global sales. Their top three games from that year are Mario Kart Wii with 35.82 million sales, Super Smash Bros. Brawl with 13.04 million sales, and Pokémon Platinum Version with 7.84 million sales.'}

In [15]:
response = chain.invoke({"query": "Determine the average sales of games by publishers who released at least two games in 2001 and compare these averages."})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:Game)-[:PUBLISHED_BY]->(p:Publisher)
WHERE g.year = 2001
WITH p, COUNT(g) AS games_count
WHERE games_count >= 2
WITH p.name AS publisher
MATCH (g:Game)-[:PUBLISHED_BY]->(p:Publisher {name: publisher})
RETURN p.name AS Publisher, AVG(g.global_sales) AS Average_Global_Sales
ORDER BY Average_Global_Sales DESC

Full Context:
[{'Publisher': 'Nintendo', 'Average_Global_Sales': 2.5413371266002818}, {'Publisher': 'Microsoft Game Studios', 'Average_Global_Sales': 1.3004761904761897}, {'Publisher': 'Enix Corporation', 'Average_Global_Sales': 1.1246666666666663}, {'Publisher': 'LucasArts', 'Average_Global_Sales': 0.9704444444444442}, {'Publisher': 'Take-Two Interactive', 'Average_Global_Sales': 0.9674092009685233}, {'Publisher': 'Sony Computer Entertainment', 'Average_Global_Sales': 0.889458272327965}, {'Publisher': 'Electronic Arts', 'Average_Global_Sales': 0.8218504811250932}, {'Publisher': 'Universal Interactive', '

{'query': 'Determine the average sales of games by publishers who released at least two games in 2001 and compare these averages.',
 'result': "Nintendo has the highest average global sales at approximately 2.54 units per game, followed by Microsoft Game Studios with about 1.30 units per game. Enix Corporation has an average of 1.12 units per game, while LucasArts has around 0.97 units per game. Take-Two Interactive's average is close to 0.97 units per game, and Sony Computer Entertainment averages about 0.89 units per game. Electronic Arts have an average sales of 0.82 units per game, Universal Interactive averages roughly 0.77 units per game, Activision's average is about 0.75 units per game, and Virgin Interactive rounds out the list with an average of 0.71 units per game."}

In [16]:
response = chain.invoke({"query": "Identify publishers who released games in both 2001 and 2002, and compare their total global sales for each year"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g2001:Game)-[:PUBLISHED_BY]->(p:Publisher)
WHERE g2001.year = 2001
WITH p, SUM(g2001.global_sales) AS sales2001
MATCH (g2002:Game)-[:PUBLISHED_BY]->(p)
WHERE g2002.year = 2002
RETURN p.name AS publisher, sales2001, SUM(g2002.global_sales) AS sales2002

Full Context:
[{'publisher': 'Nintendo', 'sales2001': 45.37000000000002, 'sales2002': 48.30999999999999}, {'publisher': 'Microsoft Game Studios', 'sales2001': 13.490000000000002, 'sales2002': 6.959999999999999}, {'publisher': 'Take-Two Interactive', 'sales2001': 20.429999999999996, 'sales2002': 20.29}, {'publisher': 'Sony Computer Entertainment', 'sales2001': 43.29, 'sales2002': 25.850000000000012}, {'publisher': 'Activision', 'sales2001': 18.590000000000003, 'sales2002': 25.939999999999994}, {'publisher': 'Ubisoft', 'sales2001': 4.99, 'sales2002': 16.21}, {'publisher': 'Electronic Arts', 'sales2001': 45.12000000000001, 'sales2002': 73.01000000000002}, {'publish

{'query': 'Identify publishers who released games in both 2001 and 2002, and compare their total global sales for each year',
 'result': "In 2001 and 2002, several publishers released games with the following total global sales:\n\n- Nintendo had sales of 45.37 million in 2001 and 48.31 million in 2002.\n- Microsoft Game Studios reported sales of 13.49 million in 2001 and 6.96 million in 2002.\n- Take-Two Interactive had sales amounting to 20.43 million in 2001 and 20.29 million in 2002.\n- Sony Computer Entertainment recorded sales of 43.29 million in 2001 and 25.85 million in 2002.\n- Activision's sales were 18.59 million in 2001 and 25.94 million in 2002.\n- Ubisoft saw sales of 4.99 million in 2001 and 16.21 million in 2002.\n- Electronic Arts had impressive sales of 45.12 million in 2001 and 73.01 million in 2002.\n- Sega reported sales of 6.60 million in 2001 and 12.30 million in 2002.\n- SquareSoft's sales were relatively low, at 0.25 million in 2001 and 0.12 million in 2002.\n-

### Reference
- https://python.langchain.com/v0.2/docs/integrations/graphs/neo4j_cypher/